1. Fine-tine SumBERT with GRUEN metric
   - investigate other metrics to use
   - 

In [ ]:
!python3 -m pip install transformers sentencepiece datasets rouge_score torch==1.10.0 -f https://download.pytorch.org/whl/lts/1.8/torch_lts.html
!python3 -m pip install wmd==1.3.1
!python -m spacy download en_core_web_md

Looking in links: https://download.pytorch.org/whl/lts/1.8/torch_lts.html
     |████████████████████████████████| 4.0 MB 4.2 MB/s 
     |████████████████████████████████| 1.2 MB 71.0 MB/s 
     |████████████████████████████████| 325 kB 69.8 MB/s 
     |████████████████████████████████| 596 kB 61.3 MB/s 
     |████████████████████████████████| 6.5 MB 55.4 MB/s 
     |████████████████████████████████| 77 kB 7.0 MB/s 
     |████████████████████████████████| 895 kB 50.4 MB/s 
     |████████████████████████████████| 212 kB 58.3 MB/s 
     |████████████████████████████████| 136 kB 89.5 MB/s 
     |████████████████████████████████| 1.1 MB 51.2 MB/s 
     |████████████████████████████████| 127 kB 93.8 MB/s 
     |████████████████████████████████| 94 kB 4.2 MB/s 
     |████████████████████████████████| 144 kB 80.4 MB/s 
     |████████████████████████████████| 271 kB 92.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import numpy as np
import pandas as pd
from transformers import *
from datasets import load_metric, Dataset
import torch as pt
from transformers import BartTokenizer, BartForConditionalGeneration
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer


Mounted at /content/drive


In [ ]:
ls

drive/  sample_data/


for marcusmanos7@gmail.com

In [ ]:
!cd drive/MyDrive/W266-final-project/


/bin/bash: line 0: cd: drive/MyDrive/W266-final-project/: No such file or directory


In [ ]:
#for marcusmanos@berkeley.edu
!cd /content/drive/My Drive/W266-final-project/

/bin/bash: line 0: cd: too many arguments


In [ ]:
GPU = 1

model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")
tokenizer = AutoTokenizer.from_pretrained("t5-small")

loading configuration file https://huggingface.co/t5-small/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/fe501e8fd6425b8ec93df37767fcce78ce626e34cc5edc859c662350cf712e41.406701565c0afd9899544c1cb8b93185a76f00b31e5ce7f6e18bbaef02241985
Model config T5Config {
  "_name_or_path": "t5-small",
  "architectures": [
    "T5WithLMHeadModel"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 6,
  "num_heads": 8,
  "num_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_lengt

## Load in the data over here

In [ ]:
data_train = pd.read_csv("train_df_indicies.csv")
data_test = pd.read_csv("test_df_indicies.csv")

In [ ]:
data_test

,Unnamed: 0,business-0,entertainment-0,politics-0,sport-0,tech-0,business-1,entertainment-1,politics-1,sport-1,...,business-28,entertainment-28,politics-28,sport-28,tech-28,business-29,entertainment-29,politics-29,sport-29,tech-29
0,0,223.0,68.0,39.0,44.0,111.0,224.0,372.0,415.0,468.0,...,33.0,44.0,299.0,67.0,385.0,317.0,370.0,188.0,247.0,298.0
1,1,372.0,83.0,6.0,70.0,253.0,52.0,54.0,144.0,496.0,...,220.0,232.0,75.0,387.0,129.0,15.0,162.0,235.0,313.0,314.0
2,2,283.0,261.0,121.0,49.0,27.0,179.0,357.0,121.0,306.0,...,206.0,108.0,220.0,281.0,85.0,2.0,167.0,13.0,506.0,204.0
3,3,108.0,331.0,323.0,303.0,267.0,132.0,41.0,187.0,26.0,...,94.0,130.0,367.0,155.0,90.0,281.0,353.0,375.0,306.0,337.0
4,4,190.0,273.0,383.0,444.0,384.0,267.0,68.0,324.0,34.0,...,75.0,86.0,181.0,263.0,165.0,16.0,89.0,395.0,167.0,338.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,481.0,61.0,217.0,193.0,307.0,439.0,248.0,40.0,312.0,...,241.0,47.0,399.0,66.0,316.0,149.0,182.0,328.0,414.0,252.0
96,96,12.0,76.0,101.0,300.0,167.0,59.0,215.0,114.0,79.0,...,167.0,182.0,338.0,397.0,361.0,276.0,355.0,4.0,428.0,259.0
97,97,48.0,255.0,61.0,232.0,381.0,114.0,376.0,264.0,371.0,...,242.0,83.0,257.0,448.0,35.0,209.0,255.0,160.0,250.0,218.0
98,98,281.0,190.0,387.0,12.0,47.0,367.0,340.0,51.0,287.0,...,27.0,290.0,319.0,228.0,307.0,402.0,94.0,414.0,97.0,55.0


In [ ]:
def get_summary(tp, num):
    path = "/content/drive/MyDrive/W266-final-project/data/BBC News Summary/Summaries/{0}/{1}.txt".format(
       tp, str(num).zfill(3))
    ret = None
    with open(path) as f:
       ret = f.read()
    return ret
  

In [ ]:
# stitch article summaries from data together, get one list with all the masked summaries available
# problem: the summaries for the fine tuning data hasn' actually been calculated yet. 
# new_data = {'type':[], 'original':[], 'masked':[], 'label':[]}

def assemble_dataset(df):
  # go thru all columns, get article type, access approppriate article from masked dataset

  ret = {'masked': [], 'original': []}
  select_types = ['business', 'entertainment', 'politics', 'sport', 'tech'] 
  # cols_masked = [c+"_masked" for c in cols]
  lbl = 0
  for col in df:
    lbl += 1
    article_type = col[:-2]

    if article_type in select_types:
      data = pd.read_csv("{0}_masked_articles.csv".format(article_type))
      for idx in df[col]:
        flag = data[article_type + " indicies"] == idx
        row = data[flag]
        try:
          article = row[article_type + " masked"].iloc[0].replace("<mask>", " <mask>")
        except Exception as e:
          
          print("MISSING ENTRY:\n [[{0}]]\n[[{1}]]".format(idx, row))
        ret['masked'].append(article)
        ret['original'].append(get_summary(article_type, int(idx)))
    if lbl % 3 == 0:
      print("{0} progress marker".format(col))
    # for j, col in row.iterrows():
    #   masked = row[col_mask].replace("<mask>", " <mask>")
    #   original = row[col + "original"]
    #   # print((summaries[col] == row[col]).unique())
    #   summary = get_summary(col, int(row[col]))
    #   new_data['article'].append(original)
    #   new_data['summary'].append(summary)

    #   lbl += 1
    
    # if i > 1:
    #   break

  return ret

In [ ]:
# train_articles = assemble_dataset(df)
i = 7
print(train_articles['masked'][i])
print(train_articles['original'][i])
test_articles = assemble_dataset(data_test)
test_articles

business masked progress marker


FileNotFoundError: ignored

In [ ]:
test_articles = assemble_dataset(data_test)
#test_articles

entertainment-0 progress marker
MISSING ENTRY:
 [[401.0]]
[[Empty DataFrame
Columns: [Unnamed: 0, tech indicies, tech masked, tech original]
Index: []]]
tech-0 progress marker
politics-1 progress marker
MISSING ENTRY:
 [[510.0]]
[[Empty DataFrame
Columns: [Unnamed: 0, business indicies, business masked, business original]
Index: []]]
business-2 progress marker
MISSING ENTRY:
 [[386.0]]
[[Empty DataFrame
Columns: [Unnamed: 0, entertainment indicies, entertainment masked, entertainment original]
Index: []]]
sport-2 progress marker
entertainment-3 progress marker
MISSING ENTRY:
 [[511.0]]
[[Empty DataFrame
Columns: [Unnamed: 0, sport indicies, sport masked, sport original]
Index: []]]
tech-3 progress marker
MISSING ENTRY:
 [[386.0]]
[[Empty DataFrame
Columns: [Unnamed: 0, entertainment indicies, entertainment masked, entertainment original]
Index: []]]
MISSING ENTRY:
 [[417.0]]
[[Empty DataFrame
Columns: [Unnamed: 0, politics indicies, politics masked, politics original]
Index: []]]
polit

In [ ]:
#go back through the missing entries and replace them with the first value, since everything was bumped up by 1, for both data_test and data_train
#find the rows with missing values/empty lists and use .apply to fill in

## GRUEN Stuff


In [ ]:
import en_core_web_md
import difflib
import editdistance
import math
import numpy as np
import re
import spacy
import string
import torch
from nltk.tokenize import sent_tokenize
from tqdm import tqdm
from transformers import BertConfig, BertForSequenceClassification, BertTokenizer, BertForMaskedLM
from transformers import glue_convert_examples_to_features
from transformers.data.processors.utils import InputExample
from wmd import WMD
import nltk
nltk.download('punkt')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


""" Processing """
def preprocess_candidates(candidates):
    for i in range(len(candidates)):
        candidates[i] = candidates[i].strip()
        candidates[i] = '. '.join(candidates[i].split('\n\n'))
        candidates[i] = '. '.join(candidates[i].split('\n'))
        candidates[i] = '.'.join(candidates[i].split('..'))
        candidates[i] = '. '.join(candidates[i].split('.'))
        candidates[i] = '. '.join(candidates[i].split('. . '))
        candidates[i] = '. '.join(candidates[i].split('.  . '))
        while len(candidates[i].split('  ')) > 1:
            candidates[i] = ' '.join(candidates[i].split('  '))
        myre = re.search(r'(\d+)\. (\d+)', candidates[i])
        while myre:
            candidates[i] = 'UNK'.join(candidates[i].split(myre.group()))
            myre = re.search(r'(\d+)\. (\d+)', candidates[i])
        candidates[i] = candidates[i].strip()
    processed_candidates = []
    for candidate_i in candidates:
        sentences = sent_tokenize(candidate_i)
        out_i = []
        for sentence_i in sentences:
            if len(sentence_i.translate(str.maketrans('', '', string.punctuation)).split()) > 1:  # More than one word.
                out_i.append(sentence_i)
        processed_candidates.append(out_i)
    return processed_candidates


""" Scores Calculation """
def get_lm_score(sentences):
    def score_sentence(sentence, tokenizer, model):
        # if len(sentence.strip().split()) <= 1:
        #     return 10000
        tokenize_input = tokenizer.tokenize(sentence)
        if len(tokenize_input) > 510:
            tokenize_input = tokenize_input[:510]
        input_ids = torch.tensor(tokenizer.encode(tokenize_input)).unsqueeze(0).to(device)
        with torch.no_grad():
            loss = model(input_ids, labels=input_ids)[0]
        return math.exp(loss.item())

    model_name = 'bert-base-cased'
    model = BertForMaskedLM.from_pretrained(model_name).to(device)
    model.eval()
    tokenizer = BertTokenizer.from_pretrained(model_name)
    lm_score = []
    for sentence in tqdm(sentences):
        if len(sentence) == 0:
            lm_score.append(0.0)
            continue
        score_i = 0.0
        for x in sentence:
            score_i += score_sentence(x, tokenizer, model)
        score_i /= len(sentence)
        lm_score.append(score_i)
    return lm_score


def get_cola_score(sentences):
    def load_pretrained_cola_model(model_name, saved_pretrained_CoLA_model_dir):
        config_class, model_class, tokenizer_class = (BertConfig, BertForSequenceClassification, BertTokenizer)
        config = config_class.from_pretrained(saved_pretrained_CoLA_model_dir, num_labels=2, finetuning_task='CoLA')
        tokenizer = tokenizer_class.from_pretrained(saved_pretrained_CoLA_model_dir, do_lower_case=0)
        model = model_class.from_pretrained(saved_pretrained_CoLA_model_dir, from_tf=bool('.ckpt' in model_name), config=config).to(device)
        model.eval()
        return tokenizer, model

    def evaluate_cola(model, candidates, tokenizer, model_name):

        def load_and_cache_examples(candidates, tokenizer):
            max_length = 128
            examples = [InputExample(guid=str(i), text_a=x) for i,x in enumerate(candidates)]
            features = glue_convert_examples_to_features(examples, tokenizer, label_list=["0", "1"], max_length=max_length, output_mode="classification")
            # Convert to Tensors and build dataset
            all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
            all_attention_mask = torch.tensor([f.attention_mask for f in features], dtype=torch.long)
            all_labels = torch.tensor([0 for f in features], dtype=torch.long)
            all_token_type_ids = torch.tensor([[0.0]*max_length for f in features], dtype=torch.long)
            dataset = torch.utils.data.TensorDataset(all_input_ids, all_attention_mask, all_token_type_ids, all_labels)
            return dataset

        eval_dataset = load_and_cache_examples(candidates, tokenizer)
        eval_dataloader = torch.utils.data.DataLoader(eval_dataset, sampler=torch.utils.data.SequentialSampler(eval_dataset), batch_size=max(1, torch.cuda.device_count()))
        preds = None
        for batch in tqdm(eval_dataloader, desc="Evaluating"):
            model.eval()
            batch = tuple(t.to(device) for t in batch)

            with torch.no_grad():
                inputs = {'input_ids': batch[0], 'attention_mask': batch[1], 'labels': batch[3]}
                if model_name.split('-')[0] != 'distilbert':
                    inputs['token_type_ids'] = batch[2] if model_name.split('-')[0] in ['bert', 'xlnet'] else None  # XLM, DistilBERT and RoBERTa don't use segment_ids
                outputs = model(**inputs)
                tmp_eval_loss, logits = outputs[:2]

            if preds is None:
                preds = logits.detach().cpu().numpy()
            else:
                preds = np.append(preds, logits.detach().cpu().numpy(), axis=0)
        return preds[:, 1].tolist()

    def convert_sentence_score_to_paragraph_score(sentence_score, sent_length):
        paragraph_score = []
        pointer = 0
        for i in sent_length:
            if i == 0:
                paragraph_score.append(0.0)
                continue
            temp_a = sentence_score[pointer:pointer + i]
            paragraph_score.append(sum(temp_a) / len(temp_a))
            pointer += i
        return paragraph_score

    model_name = 'bert-base-cased'
    saved_pretrained_CoLA_model_dir = 'cola_model/bert-base-cased/'
    tokenizer, model = load_pretrained_cola_model(model_name, saved_pretrained_CoLA_model_dir)
    candidates = [y for x in sentences for y in x]
    sent_length = [len(x) for x in sentences]
    cola_score = evaluate_cola(model, candidates, tokenizer, model_name)
    cola_score = convert_sentence_score_to_paragraph_score(cola_score, sent_length)
    return cola_score


def get_grammaticality_score(processed_candidates):
    lm_score = get_lm_score(processed_candidates)
    cola_score = get_cola_score(processed_candidates)
    grammaticality_score = [1.0 * math.exp(-0.5*x) + 1.0 * y for x, y in zip(lm_score, cola_score)]
    grammaticality_score = [max(0, x / 8.0 + 0.5) for x in grammaticality_score]  # re-scale
    return grammaticality_score


def get_redundancy_score(all_summary):
    def if_two_sentence_redundant(a, b):
        """ Determine whether there is redundancy between two sentences. """
        if a == b:
            return 4
        if (a in b) or (b in a):
            return 4
        flag_num = 0
        a_split = a.split()
        b_split = b.split()
        if max(len(a_split), len(b_split)) >= 5:
            longest_common_substring = difflib.SequenceMatcher(None, a, b).find_longest_match(0, len(a), 0, len(b))
            LCS_string_length = longest_common_substring.size
            if LCS_string_length > 0.8 * min(len(a), len(b)):
                flag_num += 1
            LCS_word_length = len(a[longest_common_substring[0]: (longest_common_substring[0]+LCS_string_length)].strip().split())
            if LCS_word_length > 0.8 * min(len(a_split), len(b_split)):
                flag_num += 1
            edit_distance = editdistance.eval(a, b)
            if edit_distance < 0.6 * max(len(a), len(b)):  # Number of modifications from the longer sentence is too small.
                flag_num += 1
            number_of_common_word = len([x for x in a_split if x in b_split])
            if number_of_common_word > 0.8 * min(len(a_split), len(b_split)):
                flag_num += 1
        return flag_num

    redundancy_score = [0.0 for x in range(len(all_summary))]
    for i in range(len(all_summary)):
        flag = 0
        summary = all_summary[i]
        if len(summary) == 1:
            continue
        for j in range(len(summary) - 1):  # for pairwise redundancy
            for k in range(j + 1, len(summary)):
                flag += if_two_sentence_redundant(summary[j].strip(), summary[k].strip())
        redundancy_score[i] += -0.1 * flag
    return redundancy_score


def get_focus_score(all_summary):
    def compute_sentence_similarity():
        nlp = en_core_web_md.load()
        nlp.add_pipe(WMD.SpacySimilarityHook(nlp), last=True)
        all_score = []
        for i in range(len(all_summary)):
            if len(all_summary[i]) == 1:
                all_score.append([1.0])
                continue
            score = []
            for j in range(1, len(all_summary[i])):
                doc1 = nlp(all_summary[i][j-1])
                doc2 = nlp(all_summary[i][j])
                try:
                    score.append(1.0/(1.0 + math.exp(-doc1.similarity(doc2)+7)))
                except:
                    score.append(1.0)
            all_score.append(score)
        return all_score

    all_score = compute_sentence_similarity()
    focus_score = [0.0 for x in range(len(all_summary))]
    for i in range(len(all_score)):
        if len(all_score[i]) == 0:
            continue
        if min(all_score[i]) < 0.05:
            focus_score[i] -= 0.1
    return focus_score


def get_gruen(candidates):
    processed_candidates = preprocess_candidates(candidates)
    grammaticality_score = get_grammaticality_score(processed_candidates)
    redundancy_score = get_redundancy_score(processed_candidates)
    focus_score = get_focus_score(processed_candidates)
    gruen_score = [min(1, max(0, sum(i))) for i in zip(grammaticality_score, redundancy_score, focus_score)]
    return gruen_score



import pandas as pd
data = pd.read_csv("bart-baseline.csv")

scores = {}
for col in ['business-results', 'sport-results', 'tech-results', 'entertainment-results']:
  gruen_score = get_gruen(list(data[col]))
  print(col + " " + str(np.mean(gruen_score)))
  scores[col] = gruen_score

## Training

In [ ]:
data_train = pd.read_csv("train_df_indicies.csv")
data_test = pd.read_csv("test_df_indicies.csv")
data_train = data_train[['business-0', 'entertainment-0', 'politics-0', 'sport-0','tech-0']]
df = pd.read_csv('business_masked_articles.csv').drop(columns=['Unnamed: 0', 'business indicies']).rename(columns={'business masked': 'masked', 'business original': 'original'})
df2 = pd.read_csv('politics_masked_articles.csv').drop(columns=['Unnamed: 0', 'politics indicies']).rename(columns={'politics masked': 'masked', 'politics original': 'original'})
df3 = pd.read_csv('sport_masked_articles.csv').drop(columns=['Unnamed: 0', 'sport indicies']).rename(columns={'sport masked': 'masked', 'sport original': 'original'})
df4  = pd.read_csv('tech_masked_articles.csv').drop(columns=['Unnamed: 0', 'tech indicies']).rename(columns={'tech masked': 'masked', 'tech original': 'original'})
df5 = pd.read_csv('entertainment_masked_articles.csv').drop(columns=['Unnamed: 0', 'entertainment indicies']).rename(columns={'entertainment masked': 'masked', 'entertainment original': 'original'})


df_train = pd.concat([df, df2, df3, df4, df5], axis=0, join="outer",
    ignore_index=False,
    keys=None,
    levels=None,
    names=None,
    verify_integrity=False,
    copy=True,)

In [ ]:
df_train

,masked,original
0,<s>Ad<mask> boost Time Warner profit\n\nQu<mas...,Ad sales boost Time Warner profit\n\nQuarterly...
1,<s><mask><mask> gains on<mask><mask> speech\n\...,Dollar gains on Greenspan speech\n\nThe dollar...
2,<s>Yukos unit buyer faces loan claim\n\nThe ow...,Yukos unit buyer faces loan claim\n\nThe owner...
3,<s>High fuel prices hit BA's profits\n\nBritis...,High fuel prices hit BA's profits\n\nBritish A...
4,<s>Pernod<mask> talk lifts Domecq\n\nShares in...,Pernod takeover talk lifts Domecq\n\nShares in...
...,...,...
380,<s>Lost Doors frontman movie<mask>\n\n<mask>or...,Lost Doors frontman movie found\n\nHistorians ...
381,<s>Last Star Wars 'not for children'\n\nThe si...,Last Star Wars 'not for children'\n\nThe sixth...
382,<s><mask> honour for<mask> Parker\n\nBritish f...,French honour for director Parker\n\nBritish f...
383,<s>Robots march to<mask> cinema summit\n\nAnim...,Robots march to US cinema summit\n\nAnimated m...


In [ ]:
#get each of the indicies for the testing, drop those rows from the test set
#test on entire set - 500 articles

train_df = pd.read_csv('business_masked_articles.csv')
pol_df = pd.read_csv('politics_masked_articles.csv')
sport_df = pd.read_csv('sport_masked_articles.csv')
tech_df  = pd.read_csv('tech_masked_articles.csv')
ent_df = pd.read_csv('entertainment_masked_articles.csv')

data_test = data_test[['business-0', 'entertainment-0', 'politics-0', 'sport-0','tech-0']]

df = pd.merge(train_df, data_test, left_on='business indicies', right_on='business-0').drop(columns=['Unnamed: 0', 'business indicies', 'business-0', 'entertainment-0', 'politics-0', 'sport-0', 'tech-0'], axis=1).rename(columns={'business masked': 'masked', 'business original': 'original'})
df2 = pd.merge(pol_df, data_test, left_on='politics indicies', right_on='politics-0').drop(columns=['Unnamed: 0', 'politics indicies', 'business-0', 'entertainment-0', 'politics-0', 'sport-0', 'tech-0'], axis=1).rename(columns={'politics masked': 'masked', 'politics original': 'original'})
df3 = pd.merge(sport_df, data_test, left_on='sport indicies', right_on='sport-0').drop(columns=['Unnamed: 0', 'sport indicies', 'business-0', 'entertainment-0', 'politics-0', 'sport-0', 'tech-0'], axis=1).rename(columns={'sport masked': 'masked', 'sport original': 'original'})
df4 = pd.merge(tech_df, data_test, left_on='tech indicies', right_on='tech-0').drop(columns=['Unnamed: 0', 'tech indicies', 'business-0', 'entertainment-0', 'politics-0', 'sport-0', 'tech-0'], axis=1).rename(columns={'tech masked': 'masked', 'tech original': 'original'})
df5 = pd.merge(ent_df, data_test, left_on='entertainment indicies', right_on='entertainment-0').drop(columns=['Unnamed: 0', 'entertainment indicies', 'business-0', 'entertainment-0', 'politics-0', 'sport-0', 'tech-0'], axis=1).rename(columns={'entertainment masked': 'masked', 'entertainment original': 'original'})

#filter out the indicies in each of the 5 categories for testing



In [ ]:
#I have the row numbers I want i just need to get them from one df to the other and delete them from the original df
df = pd.merge(train_df, data_test, left_on='business indicies', right_on='business-0')
lst = data_test['business-0'].tolist()
for i in range(len(data_test['business-0'])):
  if df['business indicies'][i] in lst:
    df.drop(df['business indicies'][i].index)
df.shape

AttributeError: ignored

In [ ]:
df.shape

(100, 9)

In [ ]:
test_masked = df_train['masked'][:100]
test_org = df_train['original'][:100]

test_dict = {'masked': test_masked, 'original': test_org}
final = df_train[df_train.index>99]
final

,masked,original
100,<s>Australia rates at four year high\n\nAustra...,Australia rates at four year high\n\nAustralia...
101,<s>US company admits Benin bribery\n\n<mask> U...,US company admits Benin bribery\n\nA US defenc...
102,"<s>US insurer Marsh cuts 2,500 jobs\n\nUp to 2...","US insurer Marsh cuts 2,500 jobs\n\nUp to 2,50..."
103,<s>US seeks new $280bn smoker ruling\n\nThe US...,US seeks new $280bn smoker ruling\n\nThe US Ju...
104,<s>Budget Aston takes on Porsche\n\nBritish ca...,Budget Aston takes on Porsche\n\nBritish car m...
...,...,...
380,<s>Lost Doors frontman movie<mask>\n\n<mask>or...,Lost Doors frontman movie found\n\nHistorians ...
381,<s>Last Star Wars 'not for children'\n\nThe si...,Last Star Wars 'not for children'\n\nThe sixth...
382,<s><mask> honour for<mask> Parker\n\nBritish f...,French honour for director Parker\n\nBritish f...
383,<s>Robots march to<mask> cinema summit\n\nAnim...,Robots march to US cinema summit\n\nAnimated m...


In [ ]:
final.to_csv('train_df_masked_all.csv')
test_dict.to_csv('test_df_masked_all.csv')

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-32-244b3bd7e248>", line 2, in <module>
    test_dict.to_csv('test_df_masked_all.csv')
AttributeError: 'dict' object has no attribute 'to_csv'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 1823, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'AttributeError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/ultratb.py", line 1132, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/usr/local/lib/python3

AttributeError: ignored

In [ ]:
L = tokenizer.model_max_length


def tokenize_function(ex):
  #print(ex)
  #print("***")
  tokens = tokenizer(ex['masked'], padding='max_length', truncation=True, max_length=L, return_tensors='pt')
  label = tokenizer(ex['original'], padding='max_length', truncation=True, max_length=L, return_tensors='pt')
  #print(tokens['attention_mask'].size())
  tokens['attention_mask'] = tokens['attention_mask'].flatten()
  tokens['input_ids'] = tokens['input_ids'].flatten()

  tokens['labels'] = label['input_ids'].flatten()

  return tokens

print('Tokenizing data. This may take a while...')
train_dataset = Dataset.from_dict(final).map(tokenize_function, writer_batch_size=1).remove_columns(['masked', 'original'])
eval_dataset = Dataset.from_dict(test_dict).map(tokenize_function, writer_batch_size=1).remove_columns(['masked', 'original'])


Tokenizing data. This may take a while...


  0%|          | 0/1720 [00:00<?, ?ex/s]

  0%|          | 0/100 [00:00<?, ?ex/s]

In [ ]:
# plan: get sample summaries done, fine tune model with rouge 

training_args = TrainingArguments(
    num_train_epochs=10,           
    per_device_train_batch_size=1, 
    per_device_eval_batch_size=1,   
    warmup_steps=500,               
    weight_decay=0.01,              
    logging_dir='./logs',
    eval_accumulation_steps=1,
    output_dir="./out")

#model.gradient_checkpointing_enable()

trainer = Trainer(
  model=model,
  args=training_args,
  train_dataset=train_dataset,
  eval_dataset=eval_dataset,
  data_collator=DefaultDataCollator(return_tensors='pt')
)

trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 1720
  Num Epochs = 10
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 17200


Step,Training Loss
500,3.218300
1000,1.155500
1500,0.974800
2000,0.888500
2500,0.875900
3000,0.848000
3500,0.843200
4000,0.780200
4500,0.837500
5000,0.786600


Saving model checkpoint to ./out/checkpoint-500
Configuration saved in ./out/checkpoint-500/config.json
Model weights saved in ./out/checkpoint-500/pytorch_model.bin
Saving model checkpoint to ./out/checkpoint-1000
Configuration saved in ./out/checkpoint-1000/config.json
Model weights saved in ./out/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to ./out/checkpoint-1500
Configuration saved in ./out/checkpoint-1500/config.json
Model weights saved in ./out/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to ./out/checkpoint-2000
Configuration saved in ./out/checkpoint-2000/config.json
Model weights saved in ./out/checkpoint-2000/pytorch_model.bin
Saving model checkpoint to ./out/checkpoint-2500
Configuration saved in ./out/checkpoint-2500/config.json
Model weights saved in ./out/checkpoint-2500/pytorch_model.bin
Saving model checkpoint to ./out/checkpoint-3000
Configuration saved in ./out/checkpoint-3000/config.json
Model weights saved in ./out/checkpoint-3000/pytorch

TrainOutput(global_step=17200, training_loss=0.8415550081119981, metrics={'train_runtime': 1659.5845, 'train_samples_per_second': 10.364, 'train_steps_per_second': 10.364, 'total_flos': 2327878985318400.0, 'train_loss': 0.8415550081119981, 'epoch': 10.0})

In [ ]:
model.save_pretrained('./experiment-T5/')

Configuration saved in ./experiment-T5/config.json
Model weights saved in ./experiment-T5/pytorch_model.bin


In [ ]:
sifted_test_articles = {i : [] for i in test_articles}
for i in range(500):
  test_tokenizer = tokenizer(test_articles['article'][i])
  if len(test_tokenizer['input_ids']) <= 1024:
    for c in test_articles:
      sifted_test_articles[c].append(test_articles[c][i])

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-45-cd54efe42668>", line 3, in <module>
    test_tokenizer = tokenizer(test_articles['article'][i])
KeyError: 'article'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 1823, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/ultratb.py", line 1132, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/usr/local/lib/python3.7/dist-packages/IPython/core

KeyError: ignored

In [ ]:
new_summarizer = pipeline("summarization", model=model, tokenizer=tokenizer, device=0)
select_types = ['business', 'entertainment', 'politics', 'sport', 'tech']
gruen_scores = {i: [] for i in select_types}
summs = new_summarizer(sifted_test_articles['article'][::20])


In [ ]:
summs

[{'summary_text': 'wembley said the deal was due to return surplus to shareholders . the sale of the US gaming division was subject to a number .'},
 {'summary_text': 'NTPC shares have risen 13 mask on its . the government has said it will use the money to feed the .'},
 {'summary_text': 'a spokesman for nortel said it had made . he said it was a "very, very, very .'},
 {'summary_text': 'a spokesman for the government said it was . "We are not going to be able to make a positive .'},
 {'summary_text': 'he will succeed Shiro Tsuda as president . he was appointed president two months ago but the business has struggled since .'},
 {'summary_text': '"we are very pleased with the figures," a BBC spokesman said . he was supposedly shot in 1989 by a man with .'},
 {'summary_text': 'Jeremy Edwards and sarah morgan were voted out . a spokesman said she was a "so .'},
 {'summary_text': 'X Factor will return for a second series . the show has been recommissioned by the BBC\'s "strict come"'},
 {'

In [ ]:
gruen_totals = {i : 0 for i in set(test_articles['type'])}
l = len(sifted_test_articles['type'][::skip]) / 5
for tp, sc in zip(sifted_test_articles['type'][::skip], bla):
  gruen_totals[tp] += sc / l

In [ ]:
gruen_totals

In [ ]:
import torch, gc
import os
gc.collect()
torch.cuda.empty_cache()

In [ ]:
df = pd.read_csv('business_df_masked_inverse_prob.csv')
df['business masked'][1]

'<s>Dollar gains on Greenspan speech<mask><mask>The dollar has hit its highest level against the euro in almost three months after the Federal Reserve head said the US trade deficit is set to stabilise.<mask><mask><mask> Alan Greenspan highlighted the US government\'s willingness to curb spending and rising household savings<mask> factors which may help<mask> reduce it. In late trading in New York, the dollar reached<mask>1.2871 against the euro<mask> from $1.2974 on Thursday. Market concerns about the deficit has hit the greenback in recent months. On Friday, Federal Reserve chairman Mr Greens<mask>\'s speech in London ahead of the meeting of G7 finance ministers sent the dollar higher after it had earlier tumbled<mask><mask> back of worse-than-expected<mask> jobs data. "I think the chairman\'s taking a much more sangu<mask><mask><mask><mask> current account deficit than he\'s taken for some time," said Robert Sinche, head of currency strategy at Bank of America in<mask> York. "He\'s 

In [ ]:
#uses the masked data, experiment 1 doesn't

#want to have the model pre trained on first 15 articles predicting the mask filling, the next 15 predicting the summaries for each article
ARTICLE = df['business masked'][1]



# T5 uses a max_length of 512 so we cut the article to 512 tokens.
inputs = tokenizer(ARTICLE, return_tensors="pt", max_length=512, truncation=True)

outputs = model.generate(
    inputs["input_ids"], max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True
)

print(tokenizer.decode(outputs[0]))

#example_english_phrase = "UN Chief Says There Is No <mask> in Syria"
#batch = tokenizer(TXT, return_tensors="pt")
#print(batch['input_ids'])
#generated_ids = model.generate(batch["input_ids"])

#model.fit(train_dataset, validation=eval_dataset)

#masked_index = (input_ids[0][11] == tokenizer.mask_token_id).nonzero().item()
#probs = logits[0, masked_index].softmax(dim=0)
#values, predictions = probs.topk(5)

#tokenizer.decode(batch['input_ids']).split()



#the test input are the articles with no masking done one them (we want to see if the model learns to mask those POS by training it on filling in masked words with distribution of the summaries)

In [ ]:
ARTICLE = '''The dollar has hit its highest level against the euro in almost three months after the Federal Reserve head said the US trade deficit is set to stabilise.China's currency remains pegged to the dollar and the US currency's sharp falls in recent months have therefore made Chinese export prices highly competitive.Market concerns about the deficit has hit the greenback in recent months."I think the chairman's taking a much more sanguine view on the current account deficit than he's taken for some time," said Robert Sinche, head of currency strategy at Bank of America in New York.The recent falls have partly been the result of big budget deficits, as well as the US's yawning current account gap, both of which need to be funded by the buying of US bonds and assets by foreign firms and governments."He's taking a longer-term view, laying out a set of conditions under which the current account deficit can improve this year and next.'''

inputs = tokenizer("summarize: " + ARTICLE, return_tensors="pt", max_length=512, truncation=True)

outputs = model.generate(
    inputs["input_ids"], max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True
)

print(tokenizer.decode(outputs[0]))

<pad> the dollar has hit its highest level against the euro in almost three months. the dollar remains pegged to the dollar and the sharp falls in recent months have made Chinese export prices highly competitive.</s>


In [ ]:
#get T5 baseline with no fine tuning


In [ ]:
new_summarizer = pipeline("summarization", model=model, tokenizer=tokenizer, device=0)

gruen_scores = []
eval_data = new_data['article'][-1:-10] # Change this line to be whatever evaluation dataset we have 
for a in eval_dataset: 
  print("***")
  summ = new_summarizer(a) 
  sc = get_gruen([summ[0]['summary_text']])
  gruen_scores.append(sc)

In [ ]:
new_summarizer(eval_data['article'][-1])

Ideas for the Future:
 - show how the GRUEN metric is a proxy for readabillity; show an unreadable summary and GRUEN score, readable summary and GRUEN score

In [ ]:
gruen_scores


# unused cells:

In [ ]:
data_train = data_train[['business-0', 'entertainment-0', 'politics-0', 'sport-0','tech-0']]
train_df = pd.read_csv('business_masked_articles.csv')
pol_df = pd.read_csv('politics_masked_articles.csv')
sport_df = pd.read_csv('sport_masked_articles.csv')
tech_df  = pd.read_csv('tech_masked_articles.csv')
ent_df = pd.read_csv('entertainment_masked_articles.csv')



#for i in data_train['business-0']: #need to get the train masked articles in one dataset
df = pd.merge(train_df, data_train, left_on='business indicies', right_on='business-0').drop(columns=['Unnamed: 0', 'business indicies', 'business-0', 'entertainment-0', 'politics-0', 'sport-0', 'tech-0'], axis=1).rename(columns={'business masked': 'masked', 'business original': 'original'})
df2 = pd.merge(pol_df, data_train, left_on='politics indicies', right_on='politics-0').drop(columns=['Unnamed: 0', 'politics indicies', 'business-0', 'entertainment-0', 'politics-0', 'sport-0', 'tech-0'], axis=1).rename(columns={'politics masked': 'masked', 'politics original': 'original'})
df3 = pd.merge(sport_df, data_train, left_on='sport indicies', right_on='sport-0').drop(columns=['Unnamed: 0', 'sport indicies', 'business-0', 'entertainment-0', 'politics-0', 'sport-0', 'tech-0'], axis=1).rename(columns={'sport masked': 'masked', 'sport original': 'original'})
df4 = pd.merge(tech_df, data_train, left_on='tech indicies', right_on='tech-0').drop(columns=['Unnamed: 0', 'tech indicies', 'business-0', 'entertainment-0', 'politics-0', 'sport-0', 'tech-0'], axis=1).rename(columns={'tech masked': 'masked', 'tech original': 'original'})
df5 = pd.merge(ent_df, data_train, left_on='entertainment indicies', right_on='entertainment-0').drop(columns=['Unnamed: 0', 'entertainment indicies', 'business-0', 'entertainment-0', 'politics-0', 'sport-0', 'tech-0'], axis=1).rename(columns={'entertainment masked': 'masked', 'entertainment original': 'original'})

df_train = pd.concat([df, df2, df3, df4, df5], axis=0)


#get the test data

data_test = data_test[['business-0', 'entertainment-0', 'politics-0', 'sport-0','tech-0']]

df = pd.merge(train_df, data_test, left_on='business indicies', right_on='business-0').drop(columns=['Unnamed: 0', 'business indicies', 'business-0', 'entertainment-0', 'politics-0', 'sport-0', 'tech-0'], axis=1).rename(columns={'business masked': 'masked', 'business original': 'original'})
df2 = pd.merge(pol_df, data_test, left_on='politics indicies', right_on='politics-0').drop(columns=['Unnamed: 0', 'politics indicies', 'business-0', 'entertainment-0', 'politics-0', 'sport-0', 'tech-0'], axis=1).rename(columns={'politics masked': 'masked', 'politics original': 'original'})
df3 = pd.merge(sport_df, data_test, left_on='sport indicies', right_on='sport-0').drop(columns=['Unnamed: 0', 'sport indicies', 'business-0', 'entertainment-0', 'politics-0', 'sport-0', 'tech-0'], axis=1).rename(columns={'sport masked': 'masked', 'sport original': 'original'})
df4 = pd.merge(tech_df, data_test, left_on='tech indicies', right_on='tech-0').drop(columns=['Unnamed: 0', 'tech indicies', 'business-0', 'entertainment-0', 'politics-0', 'sport-0', 'tech-0'], axis=1).rename(columns={'tech masked': 'masked', 'tech original': 'original'})
df5 = pd.merge(ent_df, data_test, left_on='entertainment indicies', right_on='entertainment-0').drop(columns=['Unnamed: 0', 'entertainment indicies', 'business-0', 'entertainment-0', 'politics-0', 'sport-0', 'tech-0'], axis=1).rename(columns={'entertainment masked': 'masked', 'entertainment original': 'original'})

df_test = pd.concat([df, df2, df3, df4, df5], axis=0)